# **Lab: Model Deployment**



## Exercise 1: Sklearn Pipeline

In this exercise, we will build a pipeline for transforming input data and build a Gaussian Mixture Model. We will be woking on the Mall Customer dataset:
https://raw.githubusercontent.com/AGilarde12/Mall-Customer-Segmentation/master/Data/Mall_Customers.csv

The steps are:
1.   Setup Repository
2.   Load and Explore Dataset
3.   Prepare Data
4.   Gaussian Mixture Model
5.   Build Pipeline
6.   Push Changes


### 1. Setup Repository

**[1.1]** Go to a folder of your choice on your computer (where you store projects)

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Go to a folder of your choice on your computer (where you store projects)

In [ ]:
#Solution:
cd ~/Projects/

**[1.2]** Copy the cookiecutter data science template

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Copy the cookiecutter data science template

In [ ]:
#Solution:
cookiecutter https://github.com/drivendata/cookiecutter-data-science

Follow the prompt (name the project and repo adv_dsi_lab_4)

**[1.3]** Go inside the created folder `adv_dsi_lab_4`


In [ ]:
# Placeholder for student's code (1 command line)
# Task: Go inside the created folder adv_dsi_lab_4

In [ ]:
#Solution:
cd adv_dsi_lab_4

**[1.4]** Create a file called `Dockerfile` and add the following content:

`FROM jupyter/scipy-notebook:0ce64578df46`

`RUN conda install yellowbrick`

`RUN pip install mlflow==1.13`

`RUN pip install psycopg2-binary==2.8.5`

`ENV PYTHONPATH "${PYTHONPATH}:/home/jovyan/work"`

`RUN echo "export PYTHONPATH=/home/jovyan/work" >> ~/.bashrc`

`WORKDIR /home/jovyan/work`


In [ ]:
# Placeholder for student's code (1 command line)
# Task: Create a file called Dockerfile 

In [ ]:
#Solution:
vi Dockerfile

We will create our own Docker image based on the official jupyter/scipy-notebook.

**[1.5]** Build the image from this Dockerfile

In [ ]:
docker build -t gmm-notebook:latest .

Syntax: docker build [OPTIONS] PATH 

Options:

`-t: Name and optionally a tag in the 'name:tag' format`

Documentation: https://docs.docker.com/engine/reference/commandline/build/

**[1.6]** Run the built image

In [ ]:
docker run  -dit --rm --name adv_dsi_lab_4 -p 8888:8888 -e JUPYTER_ENABLE_LAB=yes -v "$PWD":/home/jovyan/work -v ~/Projects/adv_dsi/src:/home/jovyan/work/src gmm-notebook:latest 

**[1.7]** Display last 50 lines of logs

In [ ]:
docker logs --tail 50 adv_dsi_lab_4

Copy the url displayed and paste it to a browser in order to launch Jupyter Lab

**[1.8]** Initialise the repo

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Initialise the repo

In [ ]:
#Solution:
git init

**[1.9]** Login into Github with your account (https://github.com/) and create a public repo with the name `adv_dsi_lab_4`

**[1.10]** In your local repo `adv_dsi_lab_4`, link it with Github (replace the url with your username)

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Link repo with Github

In [ ]:
#Solution:
git remote add origin git@github.com:<username>/adv_dsi_lab_1_4

**[1.11]** Add you changes to git staging area and commit them

In [ ]:
# Placeholder for student's code (2 command lines)
# Task: Add you changes to git staging area and commit them

In [ ]:
#Solution:
git add .
git commit -m "init"

**[1.12]** Push your master branch to origin

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Push your master branch to origin

In [ ]:
#Solution:
git push --set-upstream origin master

**[1.13]** Preventing push to `master` branch

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Preventing push to master branch

In [ ]:
# Solution
git config branch.master.pushRemote no_push

**[1.14]** Create a new git branch called `gmm_pipeline`

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Create a new git branch called gmm_pipeline

In [ ]:
#Solution:
git checkout -b gmm_pipeline

**[1.15]** Navigate the folder `notebooks` and create a new jupyter notebook called `1_gmm_pipeline.ipynb`

### 2.   Load and Explore Dataset

**[2.1]** Import the pandas and numpy packages

In [ ]:
# Placeholder for student's code (3 lines of Python code)
# Task: mport the pandas and numpy packages

In [ ]:
#Solution
import pandas as pd
import numpy as np

**[2.4]** Create a variable called `file_url` containing th url to the raw dataset

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Create a variable called file_url containing th url to the raw dataset

In [ ]:
#Solution
file_url = 'https://raw.githubusercontent.com/AGilarde12/Mall-Customer-Segmentation/master/Data/Mall_Customers.csv'

**[2.5]** Load the data in a dataframe called `df`


In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Load the data in a dataframe called df

In [ ]:
#Solution:
df = pd.read_csv(file_url)

**[2.6]** Display the first 5 rows of df

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Display the first 5 rows of df

In [ ]:
# Solution
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


**[2.7]** Display the dimensions (shape) of df

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Task: Display the dimensions (shape) of df

In [ ]:
# Solution
df.shape

**[2.8]** Display the summary (info) of df

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Display the summary (info) of df

In [ ]:
# Solution
df.info()

**[2.9]** Display the descriptive statistics of df


In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Display the descriptive statictics of df

In [ ]:
# Solution
df.describe()

**[2.10]** Save the dataframe locally in the `data/raw` folder

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Save the dataframe locally in the data/raw folder

In [ ]:
# Solution
df.to_csv('../data/raw/Mall_Customers.csv', index=False)

### 3. Prepare Data

**[3.1]** Create a copy of `df` and save it into a variable called `df_cleaned`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Create a copy of df and save it into a variable called df_cleaned

In [ ]:
# Solution
df_cleaned = df.copy()

**[3.2]** Drop the column `CustomerID` from the dataframe and save it into the `data/interim` folder

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Drop the column CustomerID from the dataframe and save it into the data/interim folder

In [ ]:
# Solution
df_cleaned.drop('CustomerID', axis=1, inplace=True)
df_cleaned.to_csv('../data/interim/Mall_Customers.csv', index=False)

**[3.3]** Import `train_test_split` from `sklearn.model_selection`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Import train_test_split from sklearn.model_selection

In [ ]:
# Solution
from sklearn.model_selection import train_test_split

**[3.4]** Split the data into training and testing sets with 80-20 ratio

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Split the data into training and testing sets with 80-20 ratio

In [ ]:
# Solution
X_train, X_test = train_test_split(df_cleaned, test_size=0.2, random_state=8)

**[3.5]** Create a copy of `X_train` and save it into a variable called `X_train_cleaned`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Create a copy of X_train and save it into a variable called X_train_cleaned

In [ ]:
# Solution
X_train_cleaned = X_train.copy()

**[3.6]** Import `StandardScaler` and `OneHotEncoder` from `sklearn.preprocessing`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Import StandardScaler and OneHotEncoder from sklearn.preprocessing

In [ ]:
# Solution
from sklearn.preprocessing import StandardScaler, OneHotEncoder

**[3.7]** Create a list called `num_cols` that contains `Age`, `Annual Income (k$)` and `Spending Score (1-100)`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Create a list called num_cols that contains Age, Annual Income (k$) and Spending Score (1-100)

In [ ]:
# Solution
num_cols = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']

**[3.8]** Instantiate a `StandardScaler` and called it `sc`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Instantiate a StandardScaler and called it sc

In [ ]:
# Solution
sc = StandardScaler()

**[3.9]** Fit and transform the numeric feature of `X_train_cleaned` and replace the data into it

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Fit and transform the numeric feature of X_train_cleaned and replace the data into it

In [ ]:
# Solution
X_train_cleaned[num_cols] = sc.fit_transform(X_train_cleaned[num_cols])

**[3.10]** Create a list called `cat_cols` that contains `Gender`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Create a list called cat_cols that contains Gender

In [ ]:
# Solution
cat_cols = ['Gender']

**[3.11]** Instantiate a `OneHotEncoder` with `sparse=False` and `drop='first'` and save it to a variable called `ohe`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Instantiate a OneHotEncoder with sparse=False and drop='first' and save it to a variable called ohe

In [ ]:
# Solution
ohe = OneHotEncoder(sparse=False, drop='first')

**[3.12]** Fit and transform the categorical feature of `X_train_cleaned` and replace the data into it

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Fit and transform the categorical feature of `X_train_cleaned` and replace the data into it

In [ ]:
# Solution
X_train_cleaned[cat_cols] = ohe.fit_transform(X_train_cleaned[cat_cols])

**[3.13]** Reset the indexes of `X_train_cleaned`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Reset the indexes of X_train_cleaned

In [ ]:
# Solution
X_train_cleaned.reset_index(drop=True, inplace=True)

### 4. Gaussian Mixture Model

**[4.1]** Import `KElbowVisualizer` from `yellowbrick.cluster`
`KMeans` from `sklearn.cluster`

In [ ]:
# Placeholder for student's code (2 lines of Python code)
# Task: Import KElbowVisualizer from yellowbrick.cluster KMeans from sklearn.cluster

In [ ]:
# Solution
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

**[4.2]** Instantiate a `KElbowVisualizer` with an instantied KMeans and k=(2, 15) and save it into a variable called `model`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Instantiate a KElbowVisualizer with an instantied KMeans and k=(2, 15) and save it into a variable called model

In [ ]:
# Solution
model = KElbowVisualizer(KMeans(), k=(2,15))

**[4.3]** Fit `model` with `X_train_cleaned`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Fit model with X_train_cleaned

In [ ]:
# Solution
model.fit(X_train_cleaned)

**[4.4]** Save the attribute `elbow_value_` from `model` and save it to a variable called `n_clusters`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Save the attribute elbow_value_ from model and save it to a variable called n_clusters

In [ ]:
# Solution
n_clusters = model.elbow_value_

**[4.5]** Import `GaussianMixture` from `sklearn.mixture`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Import GaussianMixture from sklearn.mixture

In [ ]:
# Solution
from sklearn.mixture import GaussianMixture

**[4.6]** Instantiate a `GaussianMixture` with `n_components` and `random_state` set and save it into a variable called `gmm`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Instantiate a GaussianMixture with n_components and random_state set and save it into a variable called gmm

In [ ]:
# Solution
gmm = GaussianMixture(n_components=n_clusters, random_state=8)

**[4.7]** Fit `gmm` with `X_train_cleaned`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Fit gmm with X_train_cleaned

In [ ]:
# Solution
gmm.fit(X_train_cleaned)

**[4.8]** Make predictions on `X_train_cleaned`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Make predictions on X_train_cleaned

In [ ]:
# Solution
gmm.predict(X_train_cleaned)

**[4.9]** Transform the first observation of `X_train_cleaned` into a dataframe and call it `obs` 

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Transform the first observation of X_train_cleaned into a dataframe and call it obs

In [ ]:
# Solution
obs = pd.DataFrame(X_train_cleaned.iloc[0]).transpose()

**[4.10]** Make prediction on `obs`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Make prediction on obs

In [ ]:
# Solution
gmm.predict(obs)

### 5. Build Pipeline

**[5.1]** Import `Pipeline` from `sklearn.pipeline`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Import Pipeline from sklearn.pipeline

In [ ]:
# Solution:
from sklearn.pipeline import Pipeline

**[5.2]** Create a `Pipeline` called `num_transformer` with one step that contains `StandardScaler`

In [ ]:
# Placeholder for student's code (multiple lines of Python code)
# Task: Create a Pipeline called num_transformer with one step that contains StandardScaler

In [ ]:
# Solution:
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

**[5.3]** Create a `Pipeline` called `cat_transformer` with one step that contains `OneHotEncoder`

In [ ]:
# Placeholder for student's code (multiple lines of Python code)
# Task: Create a Pipeline called cat_transformer with one step that contains OneHotEncoder

In [ ]:
# Solution:
cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'))
    ]
)

**[5.4]** Import `ColumnTransformer` from `sklearn.compose`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Import ColumnTransformer from sklearn.compose

In [ ]:
# Solution:
from sklearn.compose import ColumnTransformer

**[5.5]** Create a `ColumnTransformer` called `preprocessor` with 2 steps containing `num_transformer` and `cat_transformer` that will be applied respectively to `num_cols` and `cat_cols`

In [ ]:
# Placeholder for student's code (multiple lines of Python code)
# Task: Create a ColumnTransformer called preprocessor with 2 steps containing num_transformer and cat_transformer that will be applied respectively to num_cols and cat_cols

In [ ]:
# Solution:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_cols', cat_transformer, cat_cols)
    ]
)

**[5.6]** Create a `Pipeline` called `gm_pipe` that contains 2 steps `preprocessor` and another that instantiate a GaussianMixture with same parameters as previously

In [ ]:
# Placeholder for student's code (multiple lines of Python code)
# Task: Create a Pipeline called gm_pipe that contains 2 steps preprocessor and another that instantiate a GaussianMixture with same parameters as previously

In [ ]:
# Solution:
gmm_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('clustering', GaussianMixture(n_components=6, random_state=8))
    ]
)

**[5.7]** Fit `gmm_pipe` with `X_train`

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Fit gmm_pipe with X_train

In [ ]:
# Solution:
gmm_pipe.fit(X_train)

**[5.8]** Make predictions on X_train

In [ ]:
# Placeholder for student's code (1 line of Python code)
# Task: Make predictions on X_train

In [ ]:
# Solution:
gmm_pipe.predict(X_train)

**[5.9]** Transform the first observation of `X_train` into a dataframe, call it `obs` and make prediction on it

In [ ]:
# Placeholder for student's code (2 lines of Python code)
# Task: Transform the first observation of X_train into a dataframe, call it obs and make prediction on it

In [ ]:
# Solution:
obs = pd.DataFrame(X_train.iloc[0]).transpose()
gmm_pipe.predict(obs)

**[5.10]** Import `dump` from `joblib` package and save `gmm_pipe` into `models` folder

In [ ]:
# Placeholder for student's code (2 lines of Python code)
# Task: Import dump from joblib package and save gmm_pipe into models folder

In [ ]:
# Solution:
from joblib import dump 

dump(gmm_pipe,  '../models/gmm_pipeline.joblib')

### 6.   Push changes

**[6.1]** Add you changes to git staging area

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Add you changes to git staging area

In [ ]:
# Solution:
git add .

**[6.2]** Create the snapshot of your repository and add a description

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Create the snapshot of your repository and add a description

In [ ]:
# Solution:
git commit -m "gmm pipeline"

**[6.3]** Push your snapshot to Github

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Push your snapshot to Github

In [ ]:
# Solution:
git push

**[6.4]** Check out to the master branch

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Check out to the master branch

In [ ]:
# Solution:
git checkout master

**[6.5]** Pull the latest updates

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Pull the latest updates

In [ ]:
git pull

**[6.6]** Check out to the `gmm_pipeline`

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Check out to the gmm_pipeline

In [ ]:
# Solution:
git checkout gmm_pipeline

**[6.7]** Merge the `master` branch and push your changes


In [ ]:
# Placeholder for student's code (2 command lines)
# Task: Merge the master branch and push your changes

In [ ]:
# Solution:
git merge master
git push

**[6.8]** Go to Github and merge the branch after reviewing the code and fixing any conflict




**[6.9]** Stop the Docker container

In [ ]:
# Placeholder for student's code (1 command line)
# Task: Stop the Docker container

In [ ]:
# Solution:
docker stop adv_dsi_lab_4